## Codes for "Late 20th and 21st century lake energy balance response to warming in the tropical high Andes (revision for GPC)"

#### Jarunetr Sae-Lim, Bronwen Konecky, Carrie Morrill, Neal Michelutti, Christopher Grooms, and John Smol

### Abstract
The response of Andean high-alpine lakes (>4,000 m above sea level) to atmospheric warming is poorly understood, in part due to a lack of long-term limnological and meteorological observations. Here, we use in situ, reanalysis, and climate modeling output data paired with a one-dimensional lake energy balance model to investigate the response of lake thermal properties to observed and projected 20th and 21st century warming in the tropical high Andes of Peru. The lake model configuration is based on Lake Sibinacocha (13.86°S, 71.02°W, 4,860 m a.s.l.), the largest high-alpine lake in the Andes. Relationships between recent air and lake temperature changes were investigated using the model forced with 20th/21st-century ERA5-Land climate reanalysis data, and future relationships were investigated using two CMIP6 future climate scenarios with CESM2 (SSP2-4.5 and SSP5-8.5). Results show that Sibinacocha whole-lake temperature has warmed at a rate of 0.085°C/decade between 1981-2019, which is slower than other published estimates from lakes globally despite its high alpine location. Lake Sibinacocha temperatures display interannual variability that aligns with air temperature variations, suggesting that broad climatic teleconnections that affect Andean air temperatures also influence lake temperature and stratification. Under the SSP2-4.5 and SSP5-8.5 scenarios, the model indicates an acceleration of Lake Sibinacocha’s whole-lake warming rate by 4.2 to 8.6 times relative to the modern rate. By 2091-2100, Lake Sibinacocha is projected to increase 2.7°C to 6.1 °C. Lake Sibinacocha is projected to remain well-mixed throughout the 21st century which contributes to warming at all depths rather than increased stratification. Lake Sibinacocha is anticipated to respond more slowly to warming simply due to its size. Therefore, our results should be considered a conservative end-member for other lakes in the tropical high Andes, which due to their shallower sizes will likely respond more quickly to atmospheric warming.


## Input data
We use ERA5-Land reanalysis for Part 1 (calibration) and CMIP6 CESM2 SSP245, SSP585, and historical ensembles for Part 2 (simulation). All inputs were bias corrected relative to observations from nearby weather stations. 

#### 1. ERA5-Land Reanalysis (1981-2019)
We retrieved the ERA5-Land data using ERA API through cdsapi package. 

In [ ]:
import cdsapi
# https://anaconda.org/conda-forge/cdsapi
# conda install -c conda-forge cdsapi

In [ ]:
c = cdsapi.Client()

In [ ]:
c.retrieve(
    'reanalysis-era5-land', #dataset you want
    {
        'format': 'netcdf',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_dewpoint_temperature',
            '2m_temperature', 'surface_pressure', #'runoff', 
            'surface_solar_radiation_downwards', 'surface_thermal_radiation_downwards', #'total_precipitation',
        ], 
        'year': [
            '1981', '1982', '1983',
            '1984', '1985', '1986',
            '1987', '1988', '1989',
            '1990', '1991', '1992',
            '1993', '1994', '1995',
            '1996', '1997', '1998',
            '1999', '2000', '2001',
            '2002', '2003', '2004',
            '2005', '2006', '2007',
            '2008', '2009', '2010',
            '2011', '2012', '2013',
            '2014', '2015', '2016',
            '2017', '2018', '2019',
            '2020',
            
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'day':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12',
            '13','14','15',
            '16','17','18',
            '19','20','21',
            '22','23','24',
            '25','26','27',
            '28','29','30',
            '31',
        'area': [
            -13.8, -71.1, -13.8,
            -71.0,
        ], #North, West, South, East. Default: global
        'time' : [
            '00:00','01:00','02:00',
            '03:00','04:00','05:00',
            '06:00','07:00','08:00',
            '09:00','10:00','11:00',
            '12:00','13:00','14:00',
            '15:00','16:00','17:00',
            '18:00','19:00','20:00',
            '21:00','22:00','23:00',
        ],
    },
    'download.nc') #output file name.

#### 2. CMIP6 CESM2 SSP245, SSP585, historical (2015 - 2100)
I downloaded data (r4i1p1f1, r10i1p1f1, r11i1p1f1) using wget.sh. I use the shell script below to obtain .nc files.

In [ ]:
## This is a shell script. Copy this code and place in any txt reader.
## It is a loop script so that I just run once.

#!/bin/basH
for i in /RAID/home/jsae-lim/CESM/*; do #location
    echo $i
    cd $i
    for j in /$i/*; do #files in sub-folder
        echo $j
        bash $j -H
        {Enter}
        {Enter}
    done
    cd ..
done

Then I extracted, joined data, and save data in .csv.   

#### 3. Quantile mapping bias correction
The ERA5-Land reanalysis was bias corrected relative to weather station data using the quantile mapping method.

In [ ]:
#Initialization
import pandas as pd
import numpy as np
import datetime
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF
from math import sqrt

In [ ]:
# Read the ERA5-Land reanalysis variable
era = pd.read_csv('ERA5_1981-2019_updated_perutime.csv')  
era['datetime'] = pd.to_datetime(era['datetime'], format='%m/%d/%Y %H:%M')  # convert to datetime format
era['Month'] = era['datetime'].dt.month
era.head()

In [ ]:
# Read the observed variable
obs = pd.read_csv("Master_radiation_2010-2011.csv")   #longwave radiation (strd/L_down) as an example
obs['datetime'] = pd.to_datetime(obs['datetime']) 
obs.head()

In [ ]:
# Merge the reanalysis and observation using inner join
merged = pd.merge(obs[['datetime','L_down']],era[['datetime','strd']],how='inner',left_on='datetime', right_on='datetime')   
merged = merged.dropna(axis=0, how='any')
merged['Month'] = merged['datetime'].dt.month  
merged = merged.reset_index()
merged.head()

In [ ]:
## Apply quantile mapping
mod_ecdf_w = ECDF(merged['strd_cor'].loc[(merged['Month'] <= 3) | (merged['Month'] >=10)])
mod_ecdf_s = ECDF(merged['strd_cor'].loc[(merged['Month'] >= 4) & (merged['Month'] <=9)])

for j in range(len(era.datetime)):
    if (era.at[j,'Month'] <=3) or (era.at[j,'Month'] >= 10):
        p = mod_ecdf_w(era.at[j,'strd_cor']) * 100   
        corr = np.percentile(merged['L_down'].loc[(merged['Month'] <= 3) | (merged['Month'] >=10)], p) - np.percentile(
            merged['strd_cor'].loc[(merged['Month'] <= 3) | (merged['Month'] >=10)], p)      
    else:
        p = mod_ecdf_s(era.at[j,'strd_cor']) * 100   
        corr = np.percentile(merged['L_down'].loc[(merged['Month'] >= 4) & (merged['Month'] <=9)], p) - np.percentile(
            merged['strd_cor'].loc[(merged['Month'] >= 4) & (merged['Month'] <=9)], p)
    era.at[j,'qm_strd'] = era.at[j,'strd_cor'] + corr
    
era.head()
# Print
era.to_csv('qm_strd.csv')

#### 4. Delta method
The CMIP6 CESM2 SSP245 and SSP585 were corrected relative to the quantile mapped ERA5-Land using the delta method.

In [ ]:
# Initialization
import pandas as pd
import numpy as np
import datetime
import os
import glob

In [ ]:
# Import files
# We use daily quantile mapped ERA5-Land because the resolution for CESM data is daily. 
era = pd.read_csv("ERA_daily.csv") #Quantile mapped ERA5-Land
hist_10 = pd.read_csv("hist_10.csv") #historical r10i1p1f1
hist_04 = pd.read_csv("hist_04.csv") #historical r4i1p1f1
hist_11 = pd.read_csv("hist_11.csv") #historical r11i1p1f1

In [ ]:
# Prepare data
#Group to 365 days
era_year = era.groupby(by = ["Month","Day"]).mean().reset_index()
hist_10_year = hist_10.groupby(by = ["Month","Day"]).mean().reset_index()
hist_04_year = hist_04.groupby(by = ["Month","Day"]).mean().reset_index()
hist_11_year = hist_11.groupby(by = ["Month","Day"]).mean().reset_index()

#Expand to 86 years
era_expand = pd.concat([era_year]*86).reset_index()
hist_10_expand = pd.concat([hist_10_year]*86).reset_index()
hist_04_expand = pd.concat([hist_04_year]*86).reset_index()
hist_11_expand = pd.concat([hist_11_year]*86).reset_index()

In [ ]:
#Import SSP245 and 585 files and calculate for 'Delta'.
# Create a list for a certain extension.
path = r"C:\Users\...\ssp\*.csv" # type desired extension here.
path_list = glob.glob(path)
variables = ["t2m","RH","wind","ssrd","strd","sp"]

for path in path_list:
    df = pd.read_csv(path)
    df2 = df[:-1]
    df2_year = df2.groupby(["month","day"]).mean().reset_index()
    df2_expand = pd.concat([df2_year]*86).reset_index()
    delta = pd.DataFrame()
    delta["Date"] = pd.to_datetime(df["Date"])
    j = 1
    for i in variables:
        if i == "t2m":
            a = df[i]-hist_04_expand[i]
            delta[j] = a
            j=j+1
        else:
            a = df[i]/hist_04_expand[i]
            delta[j] = a
            j=j+1
    delta_hourly = delta.set_index('Date').resample("H").ffill()
    delta_hourly = delta_hourly[:-1] 
    delta_hourly = delta_hourly[~((delta_hourly.index.month == 2) & (delta_hourly.index.day == 29))]
    delta_hourly = delta_hourly.reset_index()
    delta_hourly.to_csv(path[-13:-4]+"_delta04.csv")

In [ ]:
# Calcuate to get bias corrected SSP245 and 585 files.
path = r"C:\Users\...\delta\*.csv" # type desired extension here.
path_list = glob.glob(path) # then you can use codes from 1.2 to import files.
variables = ["t2m","RH","wind","ssrd","strd","sp"]

for path in path_list:
    df = pd.read_csv(path)
    corr = pd.DataFrame()
    corr["Date"] = pd.to_datetime(df["Date"])
    j = 1
    for i in variables:
        if i == "t2m":
            a = df[str(j)]+era_hexpand[i]
            corr[i] = a
            j = j+1
        else:
            a = df[str(j)]*era_hexpand[i]
            corr[i] = a
            j = j+1
    corr.to_csv(path[-20:-4]+"_input.csv")
    
# Add date/time
path = r"C:\Users\...\input_corr\*.csv" # type desired extension here.
path_list = glob.glob(path) # then you can use codes from 1.2 to import files.

for path in path_list:
    dt = pd.read_csv("datetime.csv")
    df = pd.read_csv(path)
    dt["t2m"] = df["t2m"]
    dt["RH"] = df["RH"]
    dt["wind"] = df["wind"]
    dt["ssrd"] = df["ssrd"]
    dt["strd"] = df["strd"]
    dt["sp"] = df["sp"]
    dt.to_csv(path[-26:-4]+".csv", index=False)

## Lake model calibration
The lake model used in this project can be found here (https://github.com/carriemorrill/lake-model). The model was tuned using the bathymetric information collected from the field during 2018 and 2019 field seasons. We performed a calibration test using 1000 ensembles of ETA and CDRN parameter choices (Latin hypercube sampling) and compared relative to the lake temperature observations from Michelutti et al. (2019). Lake model parameter files can be found at lake.inc. 

In [ ]:
# Initialization
%matplotlib inline
import pandas as pd
import numpy as np

# Latin Hypercube sampling
params = pd.read_csv('lake-params.txt', delim_whitespace=True, header=None) 
params['cdrn'] = 2.e-3*params[0] + 1.e-3  #  neutral drag coefficient from 1 to 3 e-3
params['eta'] = 0.3*params[1] + 0.2       # shortwave extinction coefficient from 0.2 to 0.5
trials = range(1,1001)
params['trial']=trials
params[['cdrn','eta']].head(5)

Simulations were evaluated based on NSE, Bias, and RSR. Here is an example of indices calculation based on a single lake model simulation. We run loops to get all indices for all 1000 simulations.

In [ ]:
# Initialization
import pandas as pd
import numpy as np
import datetime

In [ ]:
# Import lake temperature observation (South end temperatures)
obs = pd.read_csv(r'C:\Users\...\SIB_south_2016-2018.csv',engine='python')  #, delim_whitespace=True) 
obs['datetime'] = pd.to_datetime(obs['Date'], format='%m/%d/%Y')  # convert to datetime format
obs['Month'] = obs['datetime'].dt.month      # create Month column from datetime
obs['Year'] = obs['datetime'].dt.year    # create Year column from datetime
obs['Day'] = obs['datetime'].dt.day
obs_day =obs.groupby(['Year','Month','Day']).mean()  # groupby Year, Month, Day and take average of groups --> daily averages
obs_day1D = np.ravel(obs_day)

In [ ]:
#Import lake temperature simulations. It has to have the same shape as the observation.
model = pd.read_csv(r'C:\Users\...\profile.dat', delim_whitespace=True, header=None,engine='python',
     skiprows=13025, skipfooter=507, usecols=(3,6,9,12,15,18,21,24,27,30)) 
model.tail()
model_1D = np.ravel(model)

In [ ]:
# Calculate values for objective functions to choose best calibration simulations
def bias(predictions, targets): 
    return (targets.mean() - predictions.mean())
def NashSut(predictions, targets):
    return (1. - ((predictions - targets) ** 2).sum()/((targets - targets.mean()) ** 2).sum())
def rsr(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).sum())/np.sqrt(((targets - targets.mean()) ** 2).sum())

In [ ]:
#Nash Sutcliffe efficiency
NashSut(model_1D,obs_day1D)                 # all depths
#NashSut(model_1D[0:351],obs_day1D[0:351])  # LST

#Bias (Observations - Model)
bias(model_1D,obs_day1D)                  # all depths
#bias(model_1D[0:351],obs_day1D[0:351])   # LST

#RSR (Root mean square error standard deviation ratio)
rsr(model_1D,obs_day1D)  # want <= 0.5    # all depths
#rsr(model_1D[0:351],obs_day1D[0:351])    # just lake surface temperature

## Visualization

#### 1. Reanalysis, raw ERA5-Land, and quantile mapped ERA5-Land comparison
Example: Figure 2a Solar radiation downward (hourly average)

In [ ]:
# Initialization
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [ ]:
# Import
df = pd.read_csv(r'Z:\Lake_model\final\Figures\data\radiation_month.csv')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6), dpi = 300)

#Raw ERA5-Land
ax.plot(df.hour, df.ssrd, linewidth=1,color= "#ff0066", alpha = 1)
ax.fill_between(df.hour, df.ssrd+df.ssrd_std, df.ssrd-df.ssrd_std, facecolor='#ff0066', alpha=0.1)

#Quantile mapped ERA5-Land
ax.plot(df.hour, df.ssrd_qm, linewidth=2,color= "#000000", alpha = 1)
ax.fill_between(df.hour, df.ssrd_qm+df.ssrd_qm_std, df.ssrd_qm-df.ssrd_qm_std, facecolor='#000000', alpha=0.1)

#Observation
ax.plot(df.hour, df.k_down, linewidth=0.8,color="#0000ff", alpha = 1, marker = "o", markersize = 3)
ax.errorbar(x = df.hour, y = df.k_down, yerr = df.k_down_std, ecolor='#0000ff', capsize=4, linewidth= 0.8,
           markeredgewidth=0.5, alpha = 0.5) 

ax.set_ylim(0,1200)# y-axis
ax.set_xlim(0,23) # x-axis (hour in a day)
ax.set_xticks([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])

fig.savefig('Figure2a.jpg', dpi = 600)

#### 2. Lake temperature-depth profile visualization
Figure 4a - 4c

In [ ]:
# Initialization
import pandas as pd
import numpy as np
import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.dates import MonthLocator, DateFormatter

In [ ]:
# Custom colormap
hexcode = ['#020024', '#02002c', '#030134', '#03023d', '#040245', '#04034d', '#050456', '#05055e', '#060566', '#06066f', '#070777', '#080880', '#071485', '#06208b', '#052c90', '#053896', '#04449b', '#0350a1', '#025ca6', '#0268ac', '#0174b1', '#0080b7', '#008dbd', '#0092b7', '#0097b1', '#009dab', '#00a2a5', '#00a79f', '#00ad99', '#00b293', '#00b78d', '#00bd87', '#00c281', '#00c87c', '#17cc76', '#2ed170', '#45d56a', '#5cda64', '#73df5e', '#8be358', '#a2e852', '#b9ed4c', '#d0f146', '#e7f640', '#fffb3b']
custom = colors.ListedColormap(hexcode)

In [ ]:
# Import lake temperature observations and simulation
# South end temperature
obs = pd.read_csv(r'C:\...\south.csv',engine='python')  #, delim_whitespace=True) 
obs['datetime'] = pd.to_datetime(obs['Date'], format='%m/%d/%Y')  # convert to datetime format
obs['Month'] = obs['datetime'].dt.month      # create Month column from datetime
obs['Year'] = obs['datetime'].dt.year    # create Year column from datetime
obs['Day'] = obs['datetime'].dt.day
obs_day =obs.groupby(['Year','Month','Day']).mean()  # groupby Year, Month, Day and take average of groups --> daily averages
obs_day1D = np.ravel(obs_day)

# North end temperature
obs2 = pd.read_csv(r'C:\...\north.csv',engine='python')  #, delim_whitespace=True) 
obs2['datetime'] = pd.to_datetime(obs2['Date'], format='%m/%d/%Y')  # convert to datetime format
obs2['Month'] = obs2['datetime'].dt.month      # create Month column from datetime
obs2['Year'] = obs2['datetime'].dt.year    # create Year column from datetime
obs2['Day'] = obs2['datetime'].dt.day
obs2_day =obs2.groupby(['Year','Month','Day']).mean()  # groupby Year, Month, Day and take average of groups --> daily averages
obs2_day1D = np.ravel(obs2_day)

# Simulated temperature
model = pd.read_csv(r'C:\...\profile.dat', delim_whitespace=True, header=None,engine='python',
model_1D = np.ravel(model)

In [ ]:
# Set up
levels = [6,6.5,7,7.5,8,8.5,9,9.5,10,10.5,11,11.5,12]
plt.rcParams.update({'font.size': 10})

month_fmt = DateFormatter('%b')
def m_fmt(x, pos=None):
    return month_fmt(x)[0]

# Visualization
fig, (ax1,ax2,ax3) = plt.subplots(3,1, figsize=(6, 12), dpi = 300)

# Figure 4a
x1 = np.arange(np.datetime64('2016-08-30'), np.datetime64('2018-08-12'))
y1 = np.array([0, -3, -6, -9, -12, -15, -18, -21, -24, -27])
X1, Y1 = np.meshgrid(x1, y1)

CS1 = ax1.contourf(X1, Y1, obs2_day[["0m","3m","6m","9m","12m","15m","18m","21m","24m","27m"]].T,levels,cmap=custom)
#ax.clabel(CS1, inline=1, fontsize=10)
ax1.set_title('Observed Sibinacocha lake temperature (north end)')

ax1.xaxis.set_major_locator(MonthLocator())
ax1.xaxis.set_major_formatter(FuncFormatter(m_fmt))
ax1.set_ylabel('Lake depth (m)')

# Figure 4b
x2 = np.arange(np.datetime64('2016-08-30'), np.datetime64('2018-08-12'))
y2 = np.array([0, -2,-5,-7,-10,-12,-15,-17,-19,-22, -27])
X2, Y2 = np.meshgrid(x2, y2)

CS2 = ax2.contourf(X2, Y2, obs_day[["0m","2.4m","4.9m","7.3m","9.8m","12.2m","14.6m","17.1m","19.5m","21.9m","27m"]].T,levels,cmap=custom)
#ax.clabel(CS1, inline=1, fontsize=10)
ax2.set_title('Observed Sibinacocha lake temperature (south end)')

ax2.xaxis.set_major_locator(MonthLocator())
ax2.xaxis.set_major_formatter(FuncFormatter(m_fmt))
ax2.set_ylabel('Lake depth (m)')

# Figure 4c
x3 = np.arange(np.datetime64('2016-08-30'), np.datetime64('2018-08-12'))
y3 = np.array([0, -3, -6, -9, -12, -15, -18, -21, -24, -27])
X3, Y3 = np.meshgrid(x3, y3)

CS3 = ax3.contourf(X3, Y3, model.T,levels, cmap=custom)
#ax.clabel(CS1, inline=1, fontsize=10)
ax3.set_title('Modeled Sibinacocha lake temperature (top 27 m)')

ax3.xaxis.set_major_locator(MonthLocator())
ax3.xaxis.set_major_formatter(FuncFormatter(m_fmt))
ax3.set_ylabel('Lake depth (m)')

#color bar
fig.subplots_adjust(bottom=0.1)
cbar_ax = fig.add_axes([0.1, 0.05, 0.8, 0.015,])
fig.colorbar(CS1, cax = cbar_ax, orientation='horizontal', label='Lake temperature (℃)' )